In [0]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [0]:
import json
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/bigFreqList.json') as json_file:
     wordsData = json.load(json_file)

In [0]:
#generate a list of words with frequency higher than the threshold
def generateWordList(threshold): 
  temp = []
  for (word, freq) in wordsData.items():
    if (freq >= threshold):
      temp.append(word)
  return temp

In [6]:
wordList = generateWordList(40)
print(wordList[-10:])
len(wordList)

['latikivn', 'imusicvn', 'ycđt', 'greatland', 'lemmevn', 'hoclamgiau', 'hpschoolvn', 'khochiasevn', 'hakiba', 'heisvn']


141836

In [0]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [8]:
for batch in batches:
  print(len(batch))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
1836


In [0]:
#The link format for vdict Viet-Eng is: https://vdict.com/m%E1%BB%99t,2,0,0.html

In [0]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [0]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",2,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"
    
    translations = {}

    #Getting the first POS tag
    firstPosTag = soup.find("div", class_ = 'phanloai')
    
    if (firstPosTag is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"
    
    curPos = firstPosTag.string.split(' \xa0')[0]

    #first translation is a tag away from the first POS tag
    curTag = firstPosTag.next_sibling.next_sibling

    #storing the number of translations a word has
    numTrans = 1

    #continue while we are either in a POS tag or a translation tag
    while (curTag != '\n'): 
        if (curTag.attrs["class"] == ["list1"]): #a translation
            temp = {} #temp dict to store each translation of a word

            #getting the raw translation
            if (curTag.find('b').string is not None): #empty translation cell
                temp['translation'] = re.split('[,;]', curTag.find('b').string)
                temp['POS'] = curPos

                #retrieving the examples of each of the translations
                try:
                    examples = curTag.findAll("ul", class_= "list2")
                    temp2 = {} #another dict to store each examples of a translation

                    numExamples = len(list(examples))
                    temp2['nums'] = numExamples

                    for j in range(0, numExamples):
                        temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                        #retrieving the usage of each examples
                        try: 
                            tempString = str(examples[j].find("li")) #extracting the usage
                            temp2['usage ' + str(j+1)] = tempString[tempString.find("<br/>")+5:-5]
                        except:
                            temp2['usage ' + str(j+1)] = "N/A"

                    temp['examples'] = temp2
                except:
                    temp['examples'] = "N/A"

                translations[numTrans] = temp #adding the translation to the dictionary

                numTrans += 1
            curTag = curTag.next_sibling                
        else: #a tag
            curPos = curTag.string.split(' \xa0')[0]
            curTag = curTag.next_sibling

    #storing the number of translations
    translations['nums'] = numTrans - 1
    return translations

In [0]:
url = "https://vdict.com/" + quote("có") + ",2,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [13]:
soup.find("div", class_ = 'phanloai').next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [14]:
from google.colab import files
start_time = time.time()

for (index, batch) in enumerate(batches):
  dictionary = {}

  #making the dictionary for each batch
  for word in batch:
      dictionary[word] = findTranslation(word)
      time.sleep(1) #wait for 1 sec between each query

  #saving the batch
  with open('vdict VI-EN batch {}.json'.format(index), 'w') as outfile:
    json.dump(dictionary, outfile)

  files.download('vdict VI-EN batch {}.json'.format(index))

print("--- %s seconds ---" % (time.time() - start_time))

Streaming output truncated to the last 5000 lines.
Finding translation for mất
Finding translation for gặp
Finding translation for minh
Not in the dictionary minh
Finding translation for khá
Finding translation for biểu
Finding translation for cái
Finding translation for nhật
Not in the dictionary nhật
Finding translation for suất
Finding translation for tín
Not in the dictionary tín
Finding translation for vui
Finding translation for toán
Finding translation for di
Finding translation for khỏe
Finding translation for âm
Finding translation for viết
Finding translation for ích
Finding translation for nghĩa
Finding translation for nâng
Finding translation for mắt
Finding translation for tranh
Finding translation for ít
Finding translation for đời
Finding translation for khoa
Finding translation for cực
Finding translation for dạng
Finding translation for tải
Finding translation for cụ
Finding translation for hãy
Finding translation for bay
Finding translation for hút
Finding translation

KeyboardInterrupt: ignored

In [0]:
findTranslation("cái")